# Notebook for retrieving and maping Water Basin Shapefiles 

# Step 0 Setup Notebook 

In [1]:
import easysnowdata
import pandas as pd
import geopandas as gpd
import ee
import numpy as np
import time
from shapely.geometry import box

# import contextily as ctx

In [2]:
ee.Authenticate()

True

# Step 1 Download All HUC 4 Geometries in US & Filter to the Region of Interest 

In [3]:
huc04_gdf = easysnowdata.hydroclimatology.get_huc_geometries(huc_level="04")

No spatial subsetting because bbox_input was not provided.


In [4]:
print(huc04_gdf.shape)
huc04_gdf.head(2)

(223, 6)


,name,huc4,areasqkm,states,tnmid,geometry
0,St. John,0101,36500.71,"CN,ME",{AAB11E45-75A7-47FE-85F1-AB3621CD80BD},"POLYGON ((-70.43221 46.43988, -70.43221 46.439..."
1,Penobscot,0102,22299.81,"CN,ME",{84FA3A11-3D62-4DF7-AA9B-044E583A5D33},"POLYGON ((-70.4183 45.7942, -70.41818 45.79408..."


In [5]:
# function that filters a gdf with a huc4 column to including ony
# those hucs in a specific huc02 area (01-17); default is PNW (17)
def filter_by_HUC02(gdf, huc_02_region="17"):
    filtered_04 = gdf[gdf["huc4"].str.startswith(huc_02_region)]
    print(
        f"There are {filtered_04.shape[0]} HUC04 regions within HUCO2 region {huc_02_region}"
    )
    # print("Full name list is:")
    # print(filtered_04["name"])
    return filtered_04

In [6]:
# Example - Find the Huc04 Regions for the PNW
huc_02_region = "16"
filtered_04 = filter_by_HUC02(huc04_gdf, huc_02_region)
filtered_04.head(2)

There are 6 HUC04 regions within HUCO2 region 16


,name,huc4,areasqkm,states,tnmid,geometry
66,Great Salt Lake,1602,74295.03,"ID,NV,UT,WY",{98264C17-0E40-4139-9A8A-516BC4A9414F},"POLYGON ((-115.00992 41.45147, -115.00985 41.4..."
146,Bear,1601,19462.57,"ID,UT,WY",{7492C028-E858-42FB-848C-A49CB81CEE4B},"POLYGON ((-112.55219 42.27961, -112.55216 42.2..."


# Step 2 Calculate bounding box for each region 

In [7]:
# function that creates an outer boundary box for a given geometry
def create_bbox(sp):
    minx, miny, maxx, maxy = sp.bounds
    bbox = box(minx, miny, maxx, maxy)
    return bbox


# function that adds a crude bounding box to each geometry in the gdf
def add_bbox(gdf):
    result = gdf.copy()
    result["bbox"] = gdf["geometry"].apply(create_bbox)
    return result

In [8]:
filtered_04_bbox = add_bbox(filtered_04)
filtered_04_bbox.head(2)

,name,huc4,areasqkm,states,tnmid,geometry,bbox
66,Great Salt Lake,1602,74295.03,"ID,NV,UT,WY",{98264C17-0E40-4139-9A8A-516BC4A9414F},"POLYGON ((-115.00992 41.45147, -115.00985 41.4...","POLYGON ((-110.88522 37.94716, -110.88522 42.4..."
146,Bear,1601,19462.57,"ID,UT,WY",{7492C028-E858-42FB-848C-A49CB81CEE4B},"POLYGON ((-112.55219 42.27961, -112.55216 42.2...","POLYGON ((-110.59848 40.71036, -110.59848 42.8..."


# Step 3 - Get lower level HUCs based on bounding box

In [9]:
# function that returns huc geometries given a bounding box and huc level (default = 10)
def get_huc(bbox, huc_04_nm, huc_04_num, huc_level="10"):
    gdf = easysnowdata.hydroclimatology.get_huc_geometries(
        bbox_input=bbox, huc_level=huc_level
    )

    # discard overinclusive entries that don't match huc_04 starting string
    huc_str = gdf.iloc[:, 1]
    idx = huc_str.str.startswith(huc_04_num)
    gdf = gdf.loc[idx]

    print(
        f"There are {gdf.shape[0]} HUC{huc_level} regions within {huc_04_nm}, huc region: {huc_04_num}"
    )
    return gdf

In [10]:
# get low level HUCs for the Puget Sound region
huc_04 = filtered_04_bbox.iloc[0]  # choose the first huc04 in the filtered region
huc_04_nm = huc_04["name"]
huc_04_num = huc_04["huc4"]
hucXX_gdf = get_huc(huc_04["bbox"], huc_04_nm, huc_04_num, huc_level="10")
hucXX_gdf.head(2)

There are 128 HUC10 regions within Great Salt Lake, huc region: 1602


,name,huc10,areasqkm,states,tnmid,geometry
16,Lower Deep Creek-Frontal Great Salt Lake,1602030904,675.34,"ID,UT",{3ED7E394-C573-4CB9-86FC-50410A3F02FD},"POLYGON ((-113.26979 41.92006, -113.2693 41.92..."
17,Jenkins Canyon-Frontal Bonneville Salt Flats,1602030616,487.62,"NV,UT",{6F524BD7-B7D2-467B-9EFC-C78FB6B2597C},"POLYGON ((-114.25315 40.7374, -114.25301 40.73..."


# Batch Processing - Putting it altogether 

In [11]:
# function that produces a gdf of all huc_X geometries within a specificed huc_02 (default is 17)
# Assumes that you have already downloaded huc04_gdf
def all_huc_geos(huc_04_gdf, huc_02_name="17", huc_level="10"):
    huc_04_filtered = filter_by_HUC02(huc04_gdf, huc_02_name).sort_values(
        by=huc04_gdf.columns[1]
    )
    filtered_04_bbox = add_bbox(huc_04_filtered)
    results = gpd.GeoDataFrame()
    for i in range(filtered_04_bbox.shape[0]):
        huc_04 = filtered_04_bbox.iloc[i]
        huc_04_nm = huc_04["name"]
        huc_04_num = huc_04["huc4"]
        print(f"Gathering the sub-regions within {huc_04_nm}, huc region: {huc_04_num}")
        hucXX_gdf = get_huc(huc_04["bbox"], huc_04_nm, huc_04_num, huc_level="10")
        if i == 0:
            results = hucXX_gdf
        else:
            results = pd.concat([results, hucXX_gdf], ignore_index=True)

    return results

In [46]:
start_time = time.time()
huc_02 = "18"
huc_level = "10"
huc10 = all_huc_geos(huc04_gdf, huc_02_name=huc_02, huc_level=huc_level)
end_time = time.time()
print(f"Time elapsed: {end_time - start_time:.2f} seconds")
huc10.head(2)

There are 10 HUC04 regions within HUCO2 region 18
Gathering the sub-regions within Klamath-Northern California Coastal, huc region: 1801
There are 147 HUC10 regions within Klamath-Northern California Coastal, huc region: 1801
Gathering the sub-regions within Sacramento, huc region: 1802
There are 188 HUC10 regions within Sacramento, huc region: 1802
Gathering the sub-regions within Tulare-Buena Vista Lakes, huc region: 1803
There are 108 HUC10 regions within Tulare-Buena Vista Lakes, huc region: 1803
Gathering the sub-regions within San Joaquin, huc region: 1804
There are 112 HUC10 regions within San Joaquin, huc region: 1804
Gathering the sub-regions within San Francisco Bay, huc region: 1805
There are 37 HUC10 regions within San Francisco Bay, huc region: 1805
Gathering the sub-regions within Central California Coastal, huc region: 1806
There are 64 HUC10 regions within Central California Coastal, huc region: 1806
Gathering the sub-regions within Southern California Coastal, huc regi

,name,huc10,areasqkm,states,tnmid,geometry
0,Alder Creek-Frontal Pacific Ocean,1801010809,760.04,CA,{A71AD21F-315A-4069-882C-AD197C90FD3E},"POLYGON ((-123.82103 39.16205, -123.8208 39.16..."
1,Salmon Creek-Frontal Pacific Ocean,1801010902,662.26,CA,{D578C388-5513-4A8C-AF69-9ADD8026D18D},"POLYGON ((-123.58388 38.72315, -123.58325 38.7..."


In [47]:
f_out = f"huc{huc_level}_in_{huc_02}"
f_out

'huc10_in_18'

In [48]:
huc10.to_file(f_out, driver="ESRI Shapefile")

# Cut file in two if too large for Github

In [49]:
def splitfiles(huc10):
    f_outA = f"huc{huc_level}_in_{huc_02}_A"
    f_outB = f"huc{huc_level}_in_{huc_02}_B"
    rows = huc10.shape[0]
    split_idx = int(rows / 2)
    huc10A = huc10.iloc[0:split_idx, :]
    huc10B = huc10.iloc[split_idx:, :]
    huc10A.to_file(f_outA, driver="ESRI Shapefile")
    huc10B.to_file(f_outB, driver="ESRI Shapefile")

In [50]:
huc10 = huc10.sort_values(by="huc10")
splitfiles(huc10)

In [51]:
f_outA = f"huc{huc_level}_in_{huc_02}_A"
f_outB = f"huc{huc_level}_in_{huc_02}_B"

In [52]:
test = gpd.read_file(f_outA)
test.tail(2)

,name,huc10,areasqkm,states,tnmid,geometry
502,Dry Creek,1804000807,280.93,CA,{7B55A498-48F2-44ED-A36F-1BD6708CD43F},"POLYGON ((-120.64566 37.47644, -120.64556 37.4..."
503,Ingalsbe Slough-Merced River,1804000808,321.38,CA,{32294535-1318-439C-821D-FCBA7DC444E8},"POLYGON ((-120.97566 37.34971, -120.97559 37.3..."


In [53]:
test2 = gpd.read_file(f_outB)
test2.head(2)

,name,huc10,areasqkm,states,tnmid,geometry
0,Headwaters Tuolumne River,1804000901,553.66,CA,{A5662066-ACCE-4142-8935-EA2D0A5BEC5F},"POLYGON ((-119.5455 37.91816, -119.54522 37.91..."
1,Rancheria Creek,1804000902,193.03,CA,{FF05F826-EFE6-466D-B2F1-E9AC7A80E33B},"POLYGON ((-119.72715 37.95363, -119.7271 37.95..."
